In [1]:
#Import packages
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [37]:
import os
import pickle
import time
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

In [113]:
# The code was removed by DSX for sharing.

In [114]:
from io import BytesIO  
import requests  
import json  
import pandas as pd

def get_data(credentials):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return json.loads(resp2.content)

my_json = get_data(credentials_1)


In [117]:
batch_size = my_json.get("batch_size")
epochs = my_json.get("epochs")

In [116]:
batch_size

64

In [16]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [17]:
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

In [18]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [23]:
# Define the model
model = Sequential()
model.add(Conv2D(48, 3, 3, border_mode='same', input_shape=(3, 32, 32,)))
model.add(Activation('relu'))
model.add(Conv2D(48, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(96, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(96, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(192, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), padding="same", input_shape=(3, 32, 32...)`
  app.launch_new_instance()
/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3))`
/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3), padding="same")`
/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3))`
/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192,

In [35]:
batch_size = 64
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

In [36]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)



Using real-time data augmentation.
Epoch 1/5
781/781 [==============================] - 640s - loss: 1.3095 - acc: 0.5296 - val_loss: 1.1968 - val_acc: 0.5706
Epoch 2/5
781/781 [==============================] - 613s - loss: 1.1952 - acc: 0.5769 - val_loss: 1.0377 - val_acc: 0.6213
Epoch 3/5
781/781 [==============================] - 683s - loss: 1.1052 - acc: 0.6099 - val_loss: 0.9276 - val_acc: 0.6751
Epoch 4/5
781/781 [==============================] - 652s - loss: 1.0400 - acc: 0.6372 - val_loss: 0.8885 - val_acc: 0.6916
Epoch 5/5
781/781 [==============================] - 625s - loss: 0.9995 - acc: 0.6523 - val_loss: 0.8158 - val_acc: 0.7135


NameError: name 'pickle' is not defined

In [38]:
with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.70
Actual Label = cat vs. Predicted Label = cat
Actual Label = ship vs. Predicted Label = ship
Actual Label = ship vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = ship
Actual Label = frog vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = frog
Actual Label = automobile vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = bird
Actual Label = cat vs. Predicted Label = cat
Actual Label = automobile vs. Predicted Label = truck
Actual Label = airplane vs. Predicted Label = cat
Actual Label = truck vs. Predicted Label = truck
Actual Label = dog vs. Predicted Label = cat
Actual Label = horse vs. Predicted Label = horse
Actual Label = truck vs. Predicted Label = truck
Actual Label = ship vs. Predicted Label = frog
Actual Label = dog vs. Predicted Label = dog
Actual Label = horse vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = ship
Actual Label = frog vs. Predicted Label = frog
Actual Label = hor

In [3]:
# fix random seed for reproducibility
seed = 11
np.random.seed(seed)

In [5]:
#concatenate the x dataset
full_dataset_x = np.concatenate((x_train, x_test), axis=0)
#random sampling of 12000 images
a = np.random.randint(0,60000,size=12000)
sample_x = full_dataset_x[a]
#concatenate the y dataset
full_dataset_y = np.concatenate((y_train, y_test), axis=0)
#random sampling of 12000 
sample_y = full_dataset_y[a]

In [41]:
# one hot encode outputs
#sample_x = np_utils.to_categorical(sample_x)
sample_y = np_utils.to_categorical(sample_y)
num_classes = sample_y.shape[1]

In [42]:
# Fit the model
model.fit(x_train, y_train, validation_data=(sample_x, sample_y), epochs=epochs, batch_size=batch_size)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 12000 samples
Epoch 1/5
50000/50000 [==============================] - 657s - loss: 0.9050 - acc: 0.6874 - val_loss: 9.5194 - val_acc: 0.4021
Epoch 2/5
50000/50000 [==============================] - 608s - loss: 0.8544 - acc: 0.7067 - val_loss: 8.0234 - val_acc: 0.4867
Epoch 3/5
50000/50000 [==============================] - 676s - loss: 0.8128 - acc: 0.7206 - val_loss: 9.2444 - val_acc: 0.4175
Epoch 4/5
50000/50000 [==============================] - 634s - loss: 0.7907 - acc: 0.7290 - val_loss: 8.8841 - val_acc: 0.4412
Epoch 5/5
50000/50000 [==============================] - 486s - loss: 0.7696 - acc: 0.7395 - val_loss: 8.8989 - val_acc: 0.4363
Accuracy: 73.73%


In [44]:
with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(sample_x, sample_y,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=sample_x.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen1 = model.predict_generator(datagen.flow(sample_x, sample_y,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=sample_x.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen1):
    actual_label = labels['label_names'][np.argmax(sample_y[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.42
Actual Label = bird vs. Predicted Label = truck
Actual Label = cat vs. Predicted Label = horse
Actual Label = horse vs. Predicted Label = horse
Actual Label = cat vs. Predicted Label = truck
Actual Label = horse vs. Predicted Label = horse
Actual Label = truck vs. Predicted Label = airplane
Actual Label = bird vs. Predicted Label = truck
Actual Label = horse vs. Predicted Label = truck
Actual Label = cat vs. Predicted Label = horse
Actual Label = deer vs. Predicted Label = ship
Actual Label = cat vs. Predicted Label = automobile
Actual Label = bird vs. Predicted Label = ship
Actual Label = ship vs. Predicted Label = horse
Actual Label = cat vs. Predicted Label = airplane
Actual Label = bird vs. Predicted Label = airplane
Actual Label = cat vs. Predicted Label = horse
Actual Label = frog vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = airplane
Actual Label = deer vs. Predicted Label = ship
Actual Label = airplane vs. Predicted Label = sh

In [45]:
predict_gen1

array([[  2.99323088e-12,   7.85449713e-35,   1.28602801e-34, ...,
          1.06914163e-01,   0.00000000e+00,   8.89191329e-01],
       [  3.68248995e-37,   0.00000000e+00,   0.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  2.46826221e-16,   1.53590787e-25,   1.20042597e-26, ...,
          0.00000000e+00,   0.00000000e+00,   6.85066695e-13],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.04061059e-19,   0.00000000e+00,   0.00000000e+00, ...,
          9.99999523e-01,   0.00000000e+00,   5.22972016e-07]], dtype=float32)

In [43]:
from sklearn.metrics import confusion_matrix

In [50]:
len(predix)

12000

In [55]:
sample_y = full_dataset_y[a]

In [60]:
actual_y=[]
for i in range(len(sample_y)):
    actual_y.append(sample_y[i].tolist())

In [66]:
type(actual_y[0])

list

In [49]:
predix=model.predict_classes(sample_x)

12000/12000 [==============================] - 76s    


In [63]:
cm = confusion_matrix(actual_y, predix, labels)

TypeError: iteration over a 0-d array

In [107]:
from scipy.stats import itemfreq

In [108]:
count= itemfreq(sample_y)

In [109]:
count

array([[   0, 1213],
       [   1, 1177],
       [   2, 1236],
       [   3, 1194],
       [   4, 1211],
       [   5, 1142],
       [   6, 1227],
       [   7, 1208],
       [   8, 1176],
       [   9, 1216]])

In [110]:
for i in range (0,len(count)):
    print(labels['label_names'][i],count[i][1])

('airplane', 1213)
('automobile', 1177)
('bird', 1236)
('cat', 1194)
('deer', 1211)
('dog', 1142)
('frog', 1227)
('horse', 1208)
('ship', 1176)
('truck', 1216)


In [93]:
sample_x.shape[0]

12000

In [101]:
xx_train.shape

(8400, 3, 32, 32)

In [97]:
from sklearn.cross_validation import train_test_split

In [99]:
xx_train, xx_test, yy_train, yy_test = train_test_split(
    sample_x, sample_y, test_size=0.30, random_state=42)

In [100]:
len(xx_train)

8400

In [83]:
xx_test = np.setdiff1d( np.arange(0,sample_x.shape[0]), xx_train)

In [84]:
len(xx_test)

3600

In [103]:
# normalize inputs from 0-255 to 0.0-1.0
xx_train = xx_train.astype('float32')
xx_test = xx_test.astype('float32')
xx_train = xx_train / 255.0
xx_test = xx_test / 255.0

In [104]:
# one hot encode outputs
yy_train = np_utils.to_categorical(yy_train)
yy_test = np_utils.to_categorical(yy_test)
num_classes = yy_test.shape[1]

In [105]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(xx_train, yy_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(xx_test, yy_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(xx_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(xx_train, yy_train,
                                     batch_size=batch_size),
                        steps_per_epoch=xx_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(xx_test, yy_test),
                        workers=4)



# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)



Using real-time data augmentation.
Epoch 1/5
131/131 [==============================] - 132s - loss: 0.8426 - acc: 0.7121 - val_loss: 0.6044 - val_acc: 0.7925
Epoch 2/5
131/131 [==============================] - 113s - loss: 0.8480 - acc: 0.7127 - val_loss: 0.5779 - val_acc: 0.7986
Epoch 3/5
131/131 [==============================] - 87s - loss: 0.8098 - acc: 0.7323 - val_loss: 0.5781 - val_acc: 0.7953
Epoch 4/5
131/131 [==============================] - 85s - loss: 0.8113 - acc: 0.7283 - val_loss: 0.5978 - val_acc: 0.7886
Epoch 5/5
131/131 [==============================] - 84s - loss: 0.8097 - acc: 0.7302 - val_loss: 0.5877 - val_acc: 0.7986


In [106]:
with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(xx_test, yy_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=xx_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen2 = model.predict_generator(datagen.flow(xx_test, yy_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=xx_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen2):
    actual_label = labels['label_names'][np.argmax(yy_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Model Accuracy = 0.77
Actual Label = horse vs. Predicted Label = deer
Actual Label = frog vs. Predicted Label = ship
Actual Label = bird vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = truck
Actual Label = bird vs. Predicted Label = deer
Actual Label = horse vs. Predicted Label = ship
Actual Label = cat vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = horse
Actual Label = ship vs. Predicted Label = dog
Actual Label = ship vs. Predicted Label = automobile
Actual Label = ship vs. Predicted Label = frog
Actual Label = horse vs. Predicted Label = dog
Actual Label = bird vs. Predicted Label = ship
Actual Label = deer vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = airplane
Actual Label = airplane vs. Predicted Label = deer
Actual Label = airplane vs. Predicted Label = automobile
Actual Label = airplane vs. Predicted Label = horse
Actual Label = horse vs. Predicted Label = horse
Actual Label = dog vs. Predicted